# Uptake Reactions
The notebook adds uptake reactions for [phenol](https://www.genome.jp/dbget-bin/www_bget?C00146), [vanillate](https://www.genome.jp/dbget-bin/www_bget?C06672), [4-hydroxybenzoate](https://www.genome.jp/dbget-bin/www_bget?C00156), [guaiacol](https://www.genome.jp/dbget-bin/www_bget?C01502), and [benzoate](https://www.genome.jp/dbget-bin/www_bget?C00180) to the *R.opacus* genome scale model.


###  Method
<ol>
    <li>Add metabolites neccessary for aromatic growth</li>
    <li>Add reactions to connect the metabolites to the metabolic network</li>
</ol>

###  Required Additional Metabolites
<ul>
    <li>Phenol: extracellular phenol and cytosolic phenol</li>
    <li>Vanillate: extracellular vanillate</li>
    <li>4-Hydroxybenzoate: none</li>
    <li>Guaiacol: extracellular guaiacol and cytosolic guaiacol</li>
    <li>Benzoate: none</li>
</ul>

###  Required Additional Reactions
<ul>
    <li>Phenol: phenol exchange, phenol transport, and phenol monooxygenase</li>
    <li>Vanillate: vanillate exchange, vanillate transport</li>
    <li>4-Hydroxybenzoate: none</li>
    <li>Guaiacol: guaiacol exchange and guaiacol transport, and guaicaol O-demethylase</li>
    <li>Benzoate: none</li>
</ul>

In [1]:
import cobra

### Load genome-scale model
Get *R.opacus* NCBI model generated by CarveMe

In [2]:
model = cobra.io.read_sbml_model("../GSMs/Ropacus_carveme_grampos.xml")
model

Name,Ropacus_carveme_grampos
Memory address,0x07f589b09ffd0
Number of metabolites,1582
Number of reactions,2382
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


### Add neccessary metabolites to the model
Note: vanillate, 4-hydroxybenzoate, and benzoate are already in the model

In [3]:
# Extracellular phenol 
phenol_e = cobra.Metabolite(
    'phenol_e',
    formula='C6H6O',
    name='phenol',
    compartment='C_e')

# Cytosolic phenol
phenol_c = cobra.Metabolite(
    'phenol_c',
    formula='C6H6O',
    name='phenol',
    compartment='C_c')

# Extracellular vanillate
vanlt_e = cobra.Metabolite(
    'vanlt_e',
    formula='C8H7O4',
    name='Vanillate',
    compartment='C_e')

# Extracellular guaiacol
guaiacol_e = cobra.Metabolite(
    'guaiacol_e',
    formula='C7H8O2',
    name='guaiacol',
    compartment='C_e')

# Cytosolic guaiacol
guaiacol_c = cobra.Metabolite(
    'guaiacol_c',
    formula='C7H8O2',
    name='guaiacol',
    compartment='C_c')

Add reactions for phenol exchange, transport, and [conversion to catechol](https://www.kegg.jp/entry/R10043) to the model

In [4]:
phenol_exchange = cobra.Reaction('EX_phenol_e')
phenol_exchange.name = 'Phenol exchange'
phenol_exchange.lower_bound = -1000.  
phenol_exchange.upper_bound = 1000.  
phenol_exchange.add_metabolites({
    phenol_e: -1.0,
})

phenol_transport = cobra.Reaction('phenol_transport')
phenol_transport.name = 'Phenol transport'
phenol_transport.lower_bound = -1000. 
phenol_transport.upper_bound = 1000. 
phenol_transport.add_metabolites({
    phenol_e: -1.0,
    phenol_c:  1.0,
}) 

# EC 1.14.13.244
phenol_monooxygenase = cobra.Reaction('PHEMOX')
phenol_monooxygenase.name = 'Phenol monooxygenase'
phenol_monooxygenase.lower_bound = 0.
phenol_monooxygenase.upper_bound = 1000.  # This is the default
phenol_monooxygenase.add_metabolites({
    phenol_c: -1.0,
    model.metabolites.get_by_id('nadh_c'):   -1.0,
    model.metabolites.get_by_id('h_c'): -1.0,
    model.metabolites.get_by_id('o2_c'): -1.0,
    model.metabolites.get_by_id('catechol_c'): 1.0,
    model.metabolites.get_by_id('nad_c'):  1.0,
    model.metabolites.get_by_id('h2o_c'):  1.0,
}) 


model.add_reactions([phenol_exchange])
model.add_reactions([phenol_transport])
model.add_reactions([phenol_monooxygenase])

Add reactions to exchange and transport vanillate

In [5]:
vanlt_exchange = cobra.Reaction('EX_vanlt_e')
vanlt_exchange.name = 'vanlt exchange'
vanlt_exchange.lower_bound = -1000.  
vanlt_exchange.upper_bound = 1000.  
vanlt_exchange.add_metabolites({
    vanlt_e: -1.0,
})

vanlt_transport = cobra.Reaction('vanlt_transport')
vanlt_transport.name = 'vanlt transport'
vanlt_transport.lower_bound = -1000. 
vanlt_transport.upper_bound = 1000. 
vanlt_transport.add_metabolites({
    vanlt_e: -1.0,
    model.metabolites.get_by_id('vanlt_c'): 1.0
}) 

model.add_reactions([vanlt_exchange])
model.add_reactions([vanlt_transport])

No reactions need to be added for hydroxybenzoate consumption

Add reactions to exchange, transport, and [conversion to catechol](https://www.genome.jp/entry/R12301) guaiacol

In [6]:
guaiacol_exchange = cobra.Reaction('EX_guaiacol_e')
guaiacol_exchange.name = 'guaiacol exchange'
guaiacol_exchange.lower_bound = -1000.  
guaiacol_exchange.upper_bound = 1000.  
guaiacol_exchange.add_metabolites({
    guaiacol_e: -1.0,
})

guaiacol_transport = cobra.Reaction('guaiacol_transport')
guaiacol_transport.name = 'guaiacol transport'
guaiacol_transport.lower_bound = -1000. 
guaiacol_transport.upper_bound = 1000. 
guaiacol_transport.add_metabolites({
    guaiacol_e: -1.0,
    guaiacol_c:  1.0,
}) 


# EC 1.14.14.-
guaiacol_o_demethylase = cobra.Reaction('GUADEM')
guaiacol_o_demethylase.name = 'guaicaol O-demethylase'
guaiacol_o_demethylase.lower_bound = 0.
guaiacol_o_demethylase.upper_bound = 1000.  # This is the default
guaiacol_o_demethylase.add_metabolites({
    guaiacol_c: -1.0,
    model.metabolites.get_by_id('nadph_c'):   -1.0,
    model.metabolites.get_by_id('o2_c'): -1.0,
    model.metabolites.get_by_id('catechol_c'): 1.0,
    model.metabolites.get_by_id('fald_c'):  1.0,
    model.metabolites.get_by_id('nadp_c'):  1.0,
    model.metabolites.get_by_id('h2o_c'):  1.0,
    
}) 

model.add_reactions([guaiacol_exchange])
model.add_reactions([guaiacol_transport])
model.add_reactions([guaiacol_o_demethylase])

No reactions need to be added for benzoate consumption

Confirm model can grow with phenol (medium composition proves no other carbon sources are consumed)

In [7]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 1.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'1 mmol of phenol can make {solution.objective_value:.3f} g of biomass')

10.0 EX_h2o_e
10.0 EX_h_e
10.0 EX_cl_e
10.0 EX_pi_e
10.0 EX_nh4_e
10.0 EX_fe3_e
10.0 EX_k_e
10.0 EX_ca2_e
10.0 EX_mg2_e
10.0 EX_mn2_e
10.0 EX_cobalt2_e
10.0 EX_zn2_e
10.0 EX_cu2_e
10.0 EX_o2_e
10.0 EX_fe2_e
10.0 EX_mobd_e
10.0 EX_so4_e
1.0 EX_phenol_e
1 mmol of phenol can make 0.084 g of biomass


Confirm model can grow with vanillate (medium composition proves no other carbon sources are consumed)

In [8]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 1.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'1 mmol of vanillate can make {solution.objective_value:.3f} g of biomass')

10.0 EX_h2o_e
10.0 EX_h_e
10.0 EX_cl_e
10.0 EX_pi_e
10.0 EX_nh4_e
10.0 EX_fe3_e
10.0 EX_k_e
10.0 EX_ca2_e
10.0 EX_mg2_e
10.0 EX_mn2_e
10.0 EX_cobalt2_e
10.0 EX_zn2_e
10.0 EX_cu2_e
10.0 EX_o2_e
10.0 EX_fe2_e
10.0 EX_mobd_e
10.0 EX_so4_e
1.0 EX_vanlt_e
1 mmol of vanillate can make 0.099 g of biomass


Confirm model can grow with 4-hydroxybenzoate (medium composition proves no other carbon sources are consumed)

In [9]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 1.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'1 mmol of 4-hydroxybenzoate can make {solution.objective_value:.3f} g of biomass')

10.0 EX_h2o_e
10.0 EX_h_e
1.0 EX_4hbz_e
10.0 EX_cl_e
10.0 EX_pi_e
10.0 EX_nh4_e
10.0 EX_fe3_e
10.0 EX_k_e
10.0 EX_ca2_e
10.0 EX_mg2_e
10.0 EX_mn2_e
10.0 EX_cobalt2_e
10.0 EX_zn2_e
10.0 EX_cu2_e
10.0 EX_o2_e
10.0 EX_fe2_e
10.0 EX_mobd_e
10.0 EX_so4_e
1 mmol of 4-hydroxybenzoate can make 0.080 g of biomass


Confirm model can grow with guaiacol (medium composition proves no other carbon sources are consumed)

In [10]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 1.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'1 mmol of guaiacol can make {solution.objective_value:.3f} g of biomass')

10.0 EX_h2o_e
10.0 EX_h_e
10.0 EX_cl_e
10.0 EX_pi_e
10.0 EX_nh4_e
10.0 EX_fe3_e
10.0 EX_k_e
10.0 EX_ca2_e
10.0 EX_mg2_e
10.0 EX_mn2_e
10.0 EX_cobalt2_e
10.0 EX_zn2_e
10.0 EX_cu2_e
10.0 EX_o2_e
10.0 EX_fe2_e
10.0 EX_mobd_e
10.0 EX_so4_e
1.0 EX_guaiacol_e
1 mmol of guaiacol can make 0.101 g of biomass


Confirm model can grow with benzoate (medium composition proves no other carbon sources are consumed)

In [11]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 1.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'1 mmol of benzoate can make {solution.objective_value:.3f} g of biomass')

10.0 EX_h2o_e
10.0 EX_h_e
10.0 EX_cl_e
10.0 EX_pi_e
10.0 EX_nh4_e
10.0 EX_fe3_e
10.0 EX_k_e
1.0 EX_bz_e
10.0 EX_ca2_e
10.0 EX_mg2_e
10.0 EX_mn2_e
10.0 EX_cobalt2_e
10.0 EX_zn2_e
10.0 EX_cu2_e
10.0 EX_o2_e
10.0 EX_fe2_e
10.0 EX_mobd_e
10.0 EX_so4_e
1 mmol of benzoate can make 0.094 g of biomass


### Add triacylglycerol production and transport reactions to the model

Add triacylglycerol metabolites to the model

In [12]:
# Cytosolic TAG
tag_c = cobra.Metabolite(
    'tag_c',
    formula='C51H98O6',
    name='triacylglycerol (16:0/16:0/16.0)',
    compartment='C_c'
)

Add [reaction](https://www.genome.jp/dbget-bin/www_bget?R02251) to produce TAG, and transport reaction

In [13]:
tag_production = cobra.Reaction('tag_production')
tag_production.name = 'triacylglycerol production'
tag_production.lower_bound = -1000. 
tag_production.upper_bound = 1000. 
tag_production.add_metabolites({
    model.metabolites.get_by_id('12dgr160_c'): -1.0, # 1,2-Diacyl-sn-glycerol (dihexadecanoyl, n-C16:0)
    model.metabolites.get_by_id('pmtcoa_c'): -1.0, # Palmitoyl-CoA (n-C16:0CoA)
    model.metabolites.get_by_id('coa_c'): 1.0,
    tag_c: 1.0,
}) 

tag_transport = cobra.Reaction('EX_tag')
tag_transport.name = 'triacylglycerol transport'
tag_transport.lower_bound = -1000. 
tag_transport.upper_bound = 1000. 
tag_transport.add_metabolites({
    tag_c: -1.0,
}) 

model.add_reactions([tag_production])
model.add_reactions([tag_transport])

In [14]:
model.reactions.get_by_id('EX_tag')

Reaction identifier,EX_tag
Name,triacylglycerol transport
Memory address,0x07f5948cd4c10
Stoichiometry,tag_c <=> triacylglycerol (16:0/16:0/16.0) <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


Confirm that model can make TAG from glucose

In [15]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 10.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    model.objective = 'EX_tag'
    solution = model.optimize()
    print(f'10 mmol of glucose can make {solution.objective_value:.5f} mmol of TAG')

10.0 EX_glc__D_e
10.0 EX_h2o_e
10.0 EX_h_e
10.0 EX_cl_e
10.0 EX_pi_e
10.0 EX_nh4_e
10.0 EX_fe3_e
10.0 EX_k_e
10.0 EX_ca2_e
10.0 EX_mg2_e
10.0 EX_mn2_e
10.0 EX_cobalt2_e
10.0 EX_zn2_e
10.0 EX_cu2_e
10.0 EX_o2_e
10.0 EX_fe2_e
10.0 EX_mobd_e
10.0 EX_so4_e
10 mmol of glucose can make 0.82759 mmol of TAG


0.82759 mmol is a reasonable maximum yield of TAG
<ul>
    <li>10 mmol of glucose can provide 60 mmol of carbon</li>
    <li>0.828 mmol of TAG requires 42.2 mmol of carbon</li>
</ul>
The unaccounted for ~18 mmol of CO2 is lost during the creation of precursors and energy molecules.

Save the updated model

In [16]:
model.id = 'ropacus_model_A'
model.name = 'R. opacus model A'
cobra.io.write_sbml_model(model, "../GSMs/ropacus_model_A.xml")
model

Name,ropacus_model_A
Memory address,0x07f589b09ffd0
Number of metabolites,1588
Number of reactions,2392
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"
